In [2]:
from pathlib import Path
import pandas as pd
import os
import shutil
import numpy as np
from pathlib import Path

In [3]:
# CELLMAPS = "~/.venv/lib/python3.8/site-packages"
CELLMAPS = Path("/home/ishang/miniconda3/envs/music/lib/python3.9/site-packages/")
REPO = Path("/home/ishang/cellmaps_toolkit/")
INPUT_FOLDER = REPO / "U2OS_ref"
# INPUT_FOLDER = "./examples/"
DATA_FOLDER = Path("/data/ishang/music/")
feature_path = DATA_FOLDER / "cell_features_test_default_cell_v1.npz"
labels = DATA_FOLDER / "cells_publicHPA.csv"
hpa_images_path = DATA_FOLDER / "IF-image.csv"

# Example Dir

In [3]:
# to run the pipeline end to end, you need your own version of the examples directory
examples_dir = Path("./examples")
baitlist = examples_dir / "baitlist.tsv"
edgelist = examples_dir / "edgelist.tsv"
samples = examples_dir / "samples.csv"
unique = examples_dir / "unique.csv"

bait_df = pd.read_csv(baitlist, sep="\t")
edge_df = pd.read_csv(edgelist, sep="\t")
sample_df = pd.read_csv(samples)
unique_df = pd.read_csv(unique)

In [4]:
bait_df.head()

,GeneSymbol,GeneID,# Interactors
0,PIK3CA,101928739,2783


In [14]:
edge_df.head()

,GeneID1,Symbol1,GeneID2,Symbol2
0,101928739,PIK3CA,219541,MED19
1,101928739,PIK3CA,26030,PLEKHG3
2,101928739,PIK3CA,129446,XIRP2
3,101928739,PIK3CA,644815,FAM83G
4,101928739,PIK3CA,23347,SMCHD1


In [15]:
sample_df.head()

,filename,if_plate_id,position,sample,status,locations,antibody,ensembl_ids,gene_names
0,/archive/7/7_C5_1_,7,C5,1,35,"Cytosol,Nuclear speckles",HPA005910,ENSG00000011007,ELOA
1,/archive/7/7_C5_2_,7,C5,2,35,"Cytosol,Nuclear speckles",HPA005910,ENSG00000011007,ELOA
2,/archive/7/7_E8_1_,7,E8,1,35,Nuclear speckles,HPA006628,ENSG00000239306,RBM14
3,/archive/7/7_E8_2_,7,E8,2,35,Nuclear speckles,HPA006628,ENSG00000239306,RBM14
4,/archive/10/10_G9_1_,10,G9,1,35,Nuclear speckles,HPA008762,ENSG00000116754,SRSF11


In [30]:
# looks like this is just a row for an antibody/ensembl_id pair, but doesn't have all the antibodies for an id
unique_df.head()

,antibody,ensembl_ids,gene_names,atlas_name,locations,n_location
0,CAB004303,ENSG00000072110,ACTN1,U-2 OS,"Focal adhesion sites,Plasma membrane",2
1,HPA046964,ENSG00000182718,ANXA2,U-2 OS,"Cytosol,Plasma membrane",2
2,HPA002564,ENSG00000100823,APEX1,U-2 OS,Nucleoplasm,1
3,HPA006550,ENSG00000111229,ARPC3,U-2 OS,Nucleoplasm,1
4,HPA042816,ENSG00000204256,BRD2,U-2 OS,Nuclear speckles,1


In [32]:
assert set(sample_df["ensembl_ids"]) == set(unique_df["ensembl_ids"])
print(len(sample_df), len(unique_df))
print(set(unique_df["atlas_name"]))
print(unique_df[unique_df["ensembl_ids"] == "ENSG00000100823"])
sample_df[sample_df["ensembl_ids"] == "ENSG00000100823"]

483 168
{'U-2 OS'}
    antibody      ensembl_ids gene_names atlas_name    locations  n_location
2  HPA002564  ENSG00000100823      APEX1     U-2 OS  Nucleoplasm           1


,filename,if_plate_id,position,sample,status,locations,antibody,ensembl_ids,gene_names
8,/archive/17/17_G2_1_,17,G2,1,35,Nucleoplasm,HPA002564,ENSG00000100823,APEX1
9,/archive/17/17_G2_2_,17,G2,2,35,Nucleoplasm,HPA002564,ENSG00000100823,APEX1
44,/archive/81/81_D2_1_,81,D2,1,35,Nucleoplasm,HPA000956,ENSG00000100823,APEX1
45,/archive/81/81_D2_2_,81,D2,2,35,Nucleoplasm,HPA000956,ENSG00000100823,APEX1
232,/archive/645/645_A1_1_,645,A1,1,35,Nucleoplasm,CAB047307,ENSG00000100823,APEX1
233,/archive/645/645_A1_3_,645,A1,3,35,Nucleoplasm,CAB047307,ENSG00000100823,APEX1
240,/archive/662/662_B11_1_,662,B11,1,35,Nucleoplasm,CAB004294,ENSG00000100823,APEX1
241,/archive/662/662_B11_2_,662,B11,2,35,Nucleoplasm,CAB004294,ENSG00000100823,APEX1


In [49]:
# indexed by ensembl_ids, collect the list of antibodies and list of locations
ensemble_ids = unique_df["ensembl_ids"].unique()
collapsed_rows = {id: {"antibodies": set(), "locations": set()} for id in ensemble_ids}

def parse_row(df_row, df_cols):
    return {df_cols[i]: df_row[1][i] for i in range(len(df_cols))}

for row in sample_df.iterrows():
    row_dict = parse_row(row, sample_df.columns)
    ensembl_id = row_dict["ensembl_ids"]
    antibody = row_dict["antibody"]
    if type(row_dict["locations"]) == float:
        continue
    locations = row_dict["locations"].split(",")
    collapsed_rows[ensembl_id]["antibodies"].add(antibody)
    collapsed_rows[ensembl_id]["locations"].update(locations)
ensembl_view_df = pd.DataFrame.from_dict(collapsed_rows, orient="index")
ensembl_view_df.head()

,antibodies,locations
ENSG00000072110,{CAB004303},"{Plasma membrane, Focal adhesion sites}"
ENSG00000182718,{HPA046964},"{Plasma membrane, Cytosol}"
ENSG00000100823,"{HPA002564, CAB047307, HPA000956, CAB004294}",{Nucleoplasm}
ENSG00000111229,{HPA006550},{Nucleoplasm}
ENSG00000204256,{HPA042816},{Nuclear speckles}


In [4]:
# get list of protein+antibody images from HPA
# get list of all proteins from bioplex
# get number of proteins with each localization in HPA and then pick one to prototype the rest with
# have a list that is the one used in the MuSIC paper
# do a robustness test (just changing antibodies randomly, maybe 100 times)
# make a hierarchy with all antibodies and get IoU of all clusters <= cutoff size
# enrich for CCD in both
# then use df to partition and drop out in that phase if bigger

In [3]:
# get csv of HPA images from the /data folder
if_images_csv = Path("~/HPA-embedding/ifimages_v23.csv")
hpa_df = pd.read_csv(if_images_csv)
print(hpa_df.columns)
hpa_df.head()

Index(['filename', 'jpg_prefix', 'if_plate_id', 'position', 'sample', 'status',
       'Image status name', 'locations', 'staining characteristics',
       'unspecific', 'antibody', 'ensembl_ids', 'gene_names', 'atlas_name',
       'versions', 'earliest_version', 'first_released', 'latest_version',
       'Spatial cell cycle', 'Intensity cell cycle', 'Annotated cell cycle',
       'gain', 'x_pos', 'y_pos', 'z_pos', 'Experiment state', 'Ab state',
       'Max tpm', 'Finished in genes', 'Protocol',
       'Gene reliability (in release)', 'Gene reliability (lims)',
       'Cell count', 'well_location_predictions_all'],
      dtype='object')


,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,z_pos,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN
2,/archive/1/1_A3_1_,https://lims.proteinatlas.org/images/2899/1_A3_1_,1,A3,1,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,2.74,ENSG00000183092,PFA,Approved,Approved,4.0,NaN
3,/archive/1/1_A3_2_,https://lims.proteinatlas.org/images/2899/1_A3_2_,1,A3,2,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,2.74,ENSG00000183092,PFA,Approved,Approved,6.0,NaN
4,/archive/1/1_A6_1_,https://lims.proteinatlas.org/images/609/1_A6_1_,1,A6,1,35,Annotated / Proteinatlas,"Endoplasmic reticulum,Nuclear membrane",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,81.38,ENSG00000102119,PFA,Enhanced,Enhanced,5.0,NaN


# Load SC Embeddings

In [9]:
CELLMAPS = "~/.venv/lib/python3.8/site-packages"
DATA_FOLDER = "/data/ishang/music/"
INPUT_FOLDER = "./examples/"
feature_path = Path("/data/ishang/music/cell_features_test_default_cell_v1.npz")
labels = Path("/data/ishang/music/cells_publicHPA.csv")
hpa_images_path = Path("/data/ishang/music/IF-image.csv")

In [10]:
labels_df = pd.read_csv(labels)
hpa_images_df = pd.read_csv(hpa_images_path)

/tmp/ipykernel_2675899/3391940461.py:2: DtypeWarning: Columns (7,8,14,16,18,19,33) have mixed types. Specify dtype option on import or set low_memory=False.
  hpa_images_df = pd.read_csv(hpa_images_path)


In [11]:
sc_image_features = np.load(feature_path)['feats']

KeyboardInterrupt: 

In [ ]:
hpa_images_df["ID"] = hpa_images_df["filename"].apply(lambda x: x.split("/")[-1][:-1])
hpa_images_df.head()

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,ID
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN,1_A1_1
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN,1_A1_2
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,5.0,NaN,1_A10_1
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,4.0,NaN,1_A10_2
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,IF_FAILED_REVIEW,IF_FAILED,0.07,NaN,PFA,NaN,NaN,3.0,NaN,1_A11_1


In [ ]:
labels_df["cell_idx"] = labels_df["ID"].astype(str) + "_" + labels_df["maskid"].astype(str)
labels_df = labels_df.join(hpa_images_df.set_index("ID"), on="ID", how="left")

In [ ]:
print(labels_df.columns)
print(labels_df["atlas_name"].unique())
u2os_indices = labels_df[labels_df["atlas_name"] == "U2OS"].index
labels_df.head()

Index(['ID', 'Label', 'maskid', 'cellmask', 'ImageWidth', 'ImageHeight',
       'Nucleoplasm', 'NuclearM', 'Nucleoli', 'NucleoliFC', 'NuclearS',
       'NuclearB', 'EndoplasmicR', 'GolgiA', 'IntermediateF', 'ActinF',
       'Microtubules', 'MitoticS', 'Centrosome', 'PlasmaM', 'Mitochondria',
       'Aggresome', 'Cytosol', 'VesiclesPCP', 'Negative', 'target', 'cell_idx',
       'filename', 'jpg_prefix', 'if_plate_id', 'position', 'sample', 'status',
       'Image status name', 'locations', 'staining characteristics',
       'unspecific', 'antibody', 'ensembl_ids', 'gene_names', 'atlas_name',
       'versions', 'earliest_version', 'first_released', 'latest_version',
       'Spatial cell cycle', 'Intensity cell cycle', 'Annotated cell cycle',
       'gain', 'x_pos', 'y_pos', 'z_pos', 'Experiment state', 'Ab state',
       'Max tpm', 'Finished in genes', 'Protocol',
       'Gene reliability (in release)', 'Gene reliability (lims)',
       'Cell count', 'well_location_predictions_all'],
   

,ID,Label,maskid,cellmask,ImageWidth,ImageHeight,Nucleoplasm,NuclearM,Nucleoli,NucleoliFC,...,z_pos,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all
0,169_E5_1,0,1,eNqFlnmXojgQwL9SCmy638zO7hw7b0abVAKCCIiACipXvv...,1728,1728,1.0,0.0,0.0,0.0,...,NaN,IF_FINISHED,IF_FINISHED,0.18,ENSG00000102057,PFA,Uncertain,Uncertain,14.0,NaN
1,169_E5_1,0,2,eNq9VvtzokgQ/pemBZOwW7tVl7vCjWCPiDx8wCCgOEjg//...,1728,1728,1.0,0.0,0.0,0.0,...,NaN,IF_FINISHED,IF_FINISHED,0.18,ENSG00000102057,PFA,Uncertain,Uncertain,14.0,NaN
2,169_E5_1,0,3,eNrFk1FPwyAQx7/S/anRxMTMV816NHuwTOpgs8tWF/3+j0...,1728,1728,1.0,0.0,0.0,0.0,...,NaN,IF_FINISHED,IF_FINISHED,0.18,ENSG00000102057,PFA,Uncertain,Uncertain,14.0,NaN
3,169_E5_1,0,4,eNp1UWlPwzAM/UtxmmnhEAjQ1FY09oDu7jZp2aGVAf//G3...,1728,1728,1.0,0.0,0.0,0.0,...,NaN,IF_FINISHED,IF_FINISHED,0.18,ENSG00000102057,PFA,Uncertain,Uncertain,14.0,NaN
4,169_E5_1,0,5,eNp1VNmS2kAM/CVrjE2lKm+pSsLYnuEmu8By2MvWcvn/36...,1728,1728,1.0,0.0,0.0,0.0,...,NaN,IF_FINISHED,IF_FINISHED,0.18,ENSG00000102057,PFA,Uncertain,Uncertain,14.0,NaN


In [11]:
u2os_embeddings = sc_image_features[u2os_indices]
u2os_labels = labels_df.loc[u2os_indices]

np.save("u2os_embeddings.npy", u2os_embeddings)
u2os_labels.to_csv("u2os_labels.csv")

# Load and check u2os sc data

In [3]:
u2os_embeddings = np.load("u2os_embeddings.npy")
u2os_labels = pd.read_csv("u2os_labels.csv")
print(len(u2os_embeddings), len(u2os_labels))

/tmp/ipykernel_2735658/2910099473.py:2: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  u2os_labels = pd.read_csv("u2os_labels.csv")


418608 418608


# select genes to include--old

In [5]:
from pathlib import Path
import shutil
import pandas as pd
import numpy as np

In [6]:
img_dir = Path(outdir_image_embedding)
img_dir_filtered = Path('./2.image_embedding_filtered')
if img_dir_filtered.exists():
    shutil.rmtree(img_dir_filtered)

ppi_dir = Path(outdir_ppi_download)
ppi_dir_filtered = Path('./1.ppi_download_filtered')
if ppi_dir_filtered.exists():
    shutil.rmtree(ppi_dir_filtered)

shutil.copytree(img_dir, img_dir_filtered)
shutil.copytree(ppi_dir, ppi_dir_filtered)

PosixPath('1.ppi_download_filtered')

In [22]:
ppi_edgelist = ppi_dir_filtered / 'ppi_edgelist.tsv'
ppi_df = pd.read_csv(ppi_edgelist, sep='\t')
ppi_df.head()

,geneA,geneB
0,PIK3CA-DT,MED19
1,PIK3CA-DT,PLEKHG3
2,PIK3CA-DT,XIRP2
3,PIK3CA-DT,FAM83G
4,PIK3CA-DT,SMCHD1


In [23]:
img_emd = img_dir_filtered / 'image_emd.tsv'
img_df = pd.read_csv(img_emd, sep='\t')
img_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
SREK1,0.028495,-0.131659,-0.140246,-0.108417,-0.100102,-0.134292,-0.042231,-0.060445,0.130666,-0.034349,...,-0.129697,0.237701,0.023639,0.173883,-0.078425,-0.094233,0.140823,0.031387,0.437817,-0.140267
TRIP12,0.120922,-0.131659,0.431988,0.099261,-0.100102,-0.134292,0.022078,-0.060445,0.003795,-0.147992,...,-0.129697,-0.013884,0.023639,0.502489,-0.078425,0.053833,0.134783,0.031387,0.304909,0.177448
MYO1B,0.022881,0.133529,-0.140246,-0.034903,-0.098910,-0.134292,0.150110,0.225479,-0.138573,0.048582,...,-0.129697,-0.013884,0.570602,-0.046523,-0.078425,0.052769,0.011603,0.031387,-0.029221,-0.140267
SRSF11,0.022881,-0.131659,0.027396,-0.108417,-0.100102,-0.134292,0.027179,-0.060445,0.151627,-0.033765,...,-0.129697,0.124886,0.023639,0.416759,-0.078425,-0.090855,0.029354,0.031387,0.667623,0.043594
FAM204A,0.022881,-0.131659,-0.140246,-0.108417,-0.073226,-0.134292,-0.042231,-0.060445,-0.138573,-0.147992,...,0.523106,0.112435,0.526269,-0.046523,0.039800,0.024477,0.011603,0.317094,0.254935,-0.140267


In [24]:
ppi_genes = set(ppi_df['geneA']) | set(ppi_df['geneB'])
img_genes = set(img_df.index)
print('Number of genes in PPI:', len(ppi_genes))
print('Number of genes in image embedding:', len(img_genes))
assert (ppi_genes & img_genes) == img_genes

gene_sample = np.random.choice(list(img_genes), int(0.9 * len(img_genes)), replace=False)
gene_sample = set(gene_sample)
gene_sample.add(ppi_df['geneA'].iloc[0])
gene_sample.add(ppi_df['geneB'].iloc[0]) # to make sure we got at least one on each side
gene_sample = list(gene_sample)
print(len(gene_sample))

Number of genes in PPI: 1010
Number of genes in image embedding: 167
152


In [25]:
print(len(ppi_df), len(img_df))
ppi_df = ppi_df[ppi_df['geneA'].isin(gene_sample) & ppi_df['geneB'].isin(gene_sample)]
gene_sample = list(set(gene_sample) & set(img_df.index))
img_df = img_df.loc[gene_sample]
print(len(ppi_df), len(img_df))

ppi_df.to_csv(ppi_edgelist, sep='\t', index=False)
img_df.to_csv(img_emd, sep='\t', index=True)

2783 167
431 150


# Gene Selection--Cell Cycle

In [4]:
# filter elements of the label and image embedding files to only include genes that are in the PPI network
# then filter both by gene selection, and pick a sc embedding for each cell
# filter the edge and baitlist going to ppi download stage
# u2os_hpa = pd.read_csv(INPUT_FOLDER / "unique.csv")
# u2os_hpa_genes = set(u2os_hpa["gene_names"].unique())
u2os_hpa_genes = set(u2os_labels["gene_names"].unique())
u2os_bioplex = pd.read_csv(INPUT_FOLDER / "edgelist.tsv", sep="\t")
u2os_bioplex_genes = set(u2os_bioplex["Symbol1"].unique()) | set(u2os_bioplex["Symbol2"].unique())
genes = u2os_hpa_genes & u2os_bioplex_genes

# PPI Download

why is this needed if we're providing a bait and edge list?

In [3]:
outdir_ppi_download = REPO / '1.ppi_download'
# edgelist = './examples/edgelist.tsv'
# baitlist = './examples/baitlist.tsv'
# provenance = './examples/provenance.json'
provenance = INPUT_FOLDER / 'provenance.json'
edgelist = INPUT_FOLDER / 'edgelist.tsv'
baitlist = INPUT_FOLDER / 'baitlist.tsv'

In [4]:
!rm -rf $outdir_ppi_download
!python $CELLMAPS/cellmaps_ppidownloader/cellmaps_ppidownloadercmd.py $outdir_ppi_download --edgelist $edgelist --baitlist $baitlist --provenance $provenance
# with open(outdir_ppi_download / 'output.log') as f:
#     for line in f:
#         print(line.replace('\\n', '\n'))

['/home/ishang/miniconda3/envs/music/lib/python3.9/site-packages/cellmaps_ppidownloader/cellmaps_ppidownloadercmd.py', '/home/ishang/cellmaps_toolkit/1.ppi_download', '--edgelist', '/home/ishang/cellmaps_toolkit/U2OS_ref/edgelist.tsv', '--baitlist', '/home/ishang/cellmaps_toolkit/U2OS_ref/baitlist.tsv', '--provenance', '/home/ishang/cellmaps_toolkit/U2OS_ref/provenance.json']
Get updated gene symbols: 100%|████████████████| 2/2 [00:21<00:00, 10.90s/steps]


# PPI Embedding

run node2vec on PPI network

Note: may want to test different p and q values

In [6]:
outdir_ppi_embedding = REPO / '2.ppi_embedding'
inputdir = REPO / '1.ppi_download'

In [7]:
!rm -rf $outdir_ppi_embedding
!python $CELLMAPS/cellmaps_ppi_embedding/cellmaps_ppi_embeddingcmd.py $outdir_ppi_embedding --inputdir $inputdir

Generating walks (CPU: 8): 100%|██████████████████| 1/1 [00:04<00:00,  4.51s/it]


# SC Embeddings

In [14]:
example_embedding_dir = REPO / "2.image_embedding/"
example_embedding_file = example_embedding_dir / "image_emd.tsv"
with open(example_embedding_file) as f:
    header = f.readline()
    embedding_dim = len(header.split("\t")) - 1
    print(repr(header))
    print(embedding_dim)

'\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\t13\t14\t15\t16\t17\t18\t19\t20\t21\t22\t23\t24\t25\t26\t27\t28\t29\t30\t31\t32\t33\t34\t35\t36\t37\t38\t39\t40\t41\t42\t43\t44\t45\t46\t47\t48\t49\t50\t51\t52\t53\t54\t55\t56\t57\t58\t59\t60\t61\t62\t63\t64\t65\t66\t67\t68\t69\t70\t71\t72\t73\t74\t75\t76\t77\t78\t79\t80\t81\t82\t83\t84\t85\t86\t87\t88\t89\t90\t91\t92\t93\t94\t95\t96\t97\t98\t99\t100\t101\t102\t103\t104\t105\t106\t107\t108\t109\t110\t111\t112\t113\t114\t115\t116\t117\t118\t119\t120\t121\t122\t123\t124\t125\t126\t127\t128\t129\t130\t131\t132\t133\t134\t135\t136\t137\t138\t139\t140\t141\t142\t143\t144\t145\t146\t147\t148\t149\t150\t151\t152\t153\t154\t155\t156\t157\t158\t159\t160\t161\t162\t163\t164\t165\t166\t167\t168\t169\t170\t171\t172\t173\t174\t175\t176\t177\t178\t179\t180\t181\t182\t183\t184\t185\t186\t187\t188\t189\t190\t191\t192\t193\t194\t195\t196\t197\t198\t199\t200\t201\t202\t203\t204\t205\t206\t207\t208\t209\t210\t211\t212\t213\t214\t215\t216\t217\t218\t219\t220\t221\t

In [15]:
outdir_imageembedding = REPO / '2.image_embedding_sc'
if not outdir_imageembedding.exists():
    outdir_imageembedding.mkdir()

sc_embeddings = []
sc_gene_symbols = []
for gene in genes:
    gene_df = u2os_labels[u2os_labels["gene_names"] == gene]
    gene_embeddings = u2os_embeddings[np.random.choice(gene_df.index, 1)][0]
    sc_embeddings.append(gene_embeddings)
    sc_gene_symbols.append(gene)

In [17]:
embedding_dim = len(sc_embeddings[0])
with open(outdir_imageembedding / "image_emd.tsv", "w") as f:
    f.write("gene\t" + "\t".join([str(i) for i in range(embedding_dim)]) + "\n")
    for i in range(len(sc_embeddings)):
        f.write(sc_gene_symbols[i] + "\t" + "\t".join([str(j) for j in sc_embeddings[i]]) + "\n")

shutil.copyfile(example_embedding_dir / "ro-crate-metadata.json", outdir_imageembedding / "ro-crate-metadata.json")

PosixPath('/home/ishang/cellmaps_toolkit/2.image_embedding_sc/ro-crate-metadata.json')

# coembedding

runs implementation of MUSE to create a single integrated embedding for each protein 

note: may want to alter # epochs to reduce training time

In [18]:
outdir_coembedding = REPO / '3.coembedding'
# ppi_embeddingdir = './2.ppi_embedding'
# image_embeddingdir = './2.image_embedding'
ppi_embeddingdir = REPO / '2.ppi_embedding'
image_embeddingdir = REPO / '2.image_embedding_sc'

In [19]:
!python $CELLMAPS/cellmaps_coembedding/cellmaps_coembeddingcmd.py $outdir_coembedding --ppi_embeddingdir $ppi_embeddingdir --image_embeddingdir $image_embeddingdir -vvvv
# !python /home/ishang/miniconda3/envs/music/lib/python3.9/site-packages/cellmaps_ppidownloader/cellmaps_ppidownloadercmd.py $outdir_coembedding --ppi_embeddingdir $ppi_embeddingdir --image_embeddingdir $image_embeddingdir

2023-11-27 03:34:29,619 DEBUG 2406ms runner.py::__init__():298 In constructor
2023-11-27 03:34:29,619 DEBUG 2407ms runner.py::run():429 In run method
Saving embedding: 0it [00:00, ?it/s]/home/ishang/miniconda3/envs/music/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 1.6028692722320557 seconds
Jaccard graph constructed in 11.871747255325317 seconds
Wrote graph to binary file in 0.023301124572753906 seconds
Running Louvain mod

# generate hierarchy

Calculates similarities between embeddings then runs HiDeF community detection to make hierarchy
Note 

In [31]:
output_hierarchy = REPO / '4.hierarchy'
coembedding_dir = REPO / '3.coembedding'

In [4]:
!python $CELLMAPS/cellmaps_generate_hierarchy/cellmaps_generate_hierarchycmd.py $output_hierarchy --coembedding_dir $coembedding_dir

python: can't open file '/cellmaps_generate_hierarchy/cellmaps_generate_hierarchycmd.py': [Errno 2] No such file or directory


# eval

In [10]:
proc_folder = "run_694421_2023_12_20" 
output_hierarchy = REPO / proc_folder / "5.hierarchy_eval"
inputdir = REPO / proc_folder / "4.hierarchy"

In [11]:
%run $CELLMAPS/cellmaps_hierarchyeval/cellmaps_hierarchyevalcmd.py $output_hierarchy --hierarchy_dir $inputdir

Generating CX
